In [23]:
import os
import pandas as pd

from pathlib import Path


In [22]:
mimic3_path = 'MIMIC-III Clinical Database 1.4'
output_path = 'data/root'

In [24]:
try:
    os.makedirs(output_path)
except:
    pass

### 先粗糙地读取mimic的csv数据

In [25]:
pats = pd.read_csv(os.path.join(mimic3_path, 'PATIENTS.csv'))
# 
pats = pats[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']]
pats.DOB = pd.to_datetime(pats.DOB)
pats.DOD = pd.to_datetime(pats.DOD)


In [26]:
pats_original = pd.read_csv(os.path.join(mimic3_path, 'PATIENTS.csv'))

In [29]:
print(pats_original.head())
print(pats.head())

   ROW_ID  SUBJECT_ID GENDER                  DOB                  DOD  \
0     234         249      F  2075-03-13 00:00:00                  NaN   
1     235         250      F  2164-12-27 00:00:00  2188-11-22 00:00:00   
2     236         251      M  2090-03-15 00:00:00                  NaN   
3     237         252      M  2078-03-06 00:00:00                  NaN   
4     238         253      F  2089-11-26 00:00:00                  NaN   

              DOD_HOSP DOD_SSN  EXPIRE_FLAG  
0                  NaN     NaN            0  
1  2188-11-22 00:00:00     NaN            1  
2                  NaN     NaN            0  
3                  NaN     NaN            0  
4                  NaN     NaN            0  
   SUBJECT_ID GENDER        DOB        DOD
0         249      F 2075-03-13        NaT
1         250      F 2164-12-27 2188-11-22
2         251      M 2090-03-15        NaT
3         252      M 2078-03-06        NaT
4         253      F 2089-11-26        NaT


In [32]:
admits = pd.read_csv(os.path.join(mimic3_path, 'ADMISSIONS.csv'))
admits = admits[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'DIAGNOSIS']]
admits.ADMITTIME = pd.to_datetime(admits.ADMITTIME)
admits.DISCHTIME = pd.to_datetime(admits.DISCHTIME)
admits.DEATHTIME = pd.to_datetime(admits.DEATHTIME)
print(admits.head())

   SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME DEATHTIME  \
0          22   165315 2196-04-09 12:26:00 2196-04-10 15:54:00       NaT   
1          23   152223 2153-09-03 07:15:00 2153-09-08 19:10:00       NaT   
2          23   124321 2157-10-18 19:34:00 2157-10-25 14:00:00       NaT   
3          24   161859 2139-06-06 16:14:00 2139-06-09 12:48:00       NaT   
4          25   129635 2160-11-02 02:06:00 2160-11-05 14:55:00       NaT   

  ETHNICITY                                          DIAGNOSIS  
0     WHITE                            BENZODIAZEPINE OVERDOSE  
1     WHITE  CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...  
2     WHITE                                         BRAIN MASS  
3     WHITE                     INTERIOR MYOCARDIAL INFARCTION  
4     WHITE                            ACUTE CORONARY SYNDROME  


In [33]:
stays = pd.read_csv(os.path.join(mimic3_path, 'ICUSTAYS.csv'))
stays.INTIME = pd.to_datetime(stays.INTIME)
stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
print(stays.head())

   ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID DBSOURCE FIRST_CAREUNIT  \
0     365         268   110404      280836  carevue           MICU   
1     366         269   106296      206613  carevue           MICU   
2     367         270   188028      220345  carevue            CCU   
3     368         271   173727      249196  carevue           MICU   
4     369         272   164716      210407  carevue            CCU   

  LAST_CAREUNIT  FIRST_WARDID  LAST_WARDID              INTIME  \
0          MICU            52           52 2198-02-14 23:27:38   
1          MICU            52           52 2170-11-05 11:05:29   
2           CCU            57           57 2128-06-24 15:05:20   
3          SICU            52           23 2120-08-07 23:12:42   
4           CCU            57           57 2186-12-25 21:08:04   

              OUTTIME     LOS  
0 2198-02-18 05:26:11  3.2490  
1 2170-11-08 17:46:57  3.2788  
2 2128-06-27 12:32:29  2.8939  
3 2120-08-10 00:39:04  2.0600  
4 2186-12-27 12:01:13 

In [34]:
print('START:\n\tICUSTAY_IDs: {}\n\tHADM_IDs: {}\n\tSUBJECT_IDs: {}'.format(stays.ICUSTAY_ID.unique().shape[0],
          stays.HADM_ID.unique().shape[0], stays.SUBJECT_ID.unique().shape[0]))


START:
	ICUSTAY_IDs: 61532
	HADM_IDs: 57786
	SUBJECT_IDs: 46476
